In [ ]:
from qdrant_client import QdrantClient, models
import numpy as np
import requests
import uuid

In [ ]:
from openai import OpenAI

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [ ]:
documents = []

for course in documents_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [ ]:
# client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
qd_client = QdrantClient(url="http://localhost:6333")

In [ ]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = 'zoomcamp-sparse'

In [ ]:
qd_client.delete_collection(collection_name=collection_name)

In [ ]:
collection_name = 'zoomcamp-sparse'
qd_client.create_collection(
    collection_name=collection_name,
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

In [ ]:
info = qd_client.get_collection(collection_name)
print(info)

In [ ]:
# qd_client.create_payload_index(
#     collection_name=collection_name,
#     field_name="course",
#     field_schema="keyword"
# )

In [ ]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id = i,
        vector = vector ,
        payload=doc
    )
    points.append(point)


In [ ]:
# Send the points to the collection
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
def vector_search(query):
    course = "data-engineering-zoomcamp"
    query_points=qd_client.query_points(
        collection_name = collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key='course',
                    match = models.MatchValue(value=course)
                )
            ]
        ),
        limit=1,
        with_payload=True
    )
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """ 
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE

        QUESTION: {question}

        CONTEXT: 
        {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content 

In [ ]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = 'the course has already started, can i still enroll?'

In [ ]:
rag(query)